# Single Particle performance plots


   * [GEN Level plots](#Gen-level)
   * [All clusters without selection](#All-clusters-without-selection)
      * [2D clusters per layer](#2D-clusters-per-layer)
      * [2D cluster energy](#2D-cluster-energy)
      * [2D cluster: # of TriggerCells](#2D-cluster:-#-of-TriggerCells)
      * [# of 3D clusters](##-of-3D-clusters)
   * [Cluster matched to GEN particles](#Cluster-matched-to-GEN-particles)
      * [2D clusters per layer](#2D-clusters-per-layer)
      * [2D clusters: # of Trigger Cells](#2D-clusters:-#-of-Trigger-Cells)
      * [2D Cluster Energy](#2D-Cluster-Energy)
      * [3D clusters: # of 2D components](#3D-clusters:-#-of-2D-components)
      * [3D cluster energy](#3D-cluster-energy)
   * [Resolution studies](#Resolution-Studies)
      * [3D cluster: pt resolution](#3D-cluster:-Pt-resolution)
      * [2D position resolution](#Postion-resolution-of-2D-clusters)
   * [Trigger Tower performance](#Trigger-Tower-plots)
      * [Towers matched to GEN](#Towers-matched-to-GEN-particles)
      * [Resolution studies](#Trigger-Tower-resolution)



In [17]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
     

In [18]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def drawAll(histograms,
            labels=None,
            options='',
            text=None,
            norm=False,
            logy=False,
            min_y=None,
            max_y=None,
            y_axis_label=None,
            do_ratio=False,
            do_profile=False):
    if len(histograms) == 0:
        print 'ERROR: no histogram in input'
        return -1
    if do_ratio and len(histograms) != 2:
        print 'ERROR: do_ratio option only available when 2 histograms need to be drawn!'
        return -2
    if do_profile and not ('TH2' in histograms[0].ClassName() or 'TH3' in histograms[0].ClassName()):
        print 'ERROR: do_profile option only available with TH2 and TH3 histograms'
        return -3

    # we build the canvas
    return


def drawAll(histograms,
            labels=None,
            options='',
            text=None,
            norm=False,
            logy=False,
            min_y=None,
            max_y=None,
            y_axis_label=None,
            do_ratio=False,
            do_profile=False):
    if len(histograms) == 0:
        print 'ERROR: no histogram in input'
        return -1
    if do_ratio and len(histograms) != 2:
        print 'ERROR: do_ratio option only available when 2 histograms need to be drawn!'
        return -2
    if do_profile and not ('TH2' in histograms[0].ClassName() or 'TH3' in histograms[0].ClassName()):
        print 'ERROR: do_profile option only available with TH2 and TH3 histograms'
        return -3

    # we build the canvas
    return 0


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    if h_lines:
        for h_line_y in h_lines:
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min, max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1, "gaus", minfit, maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")


def drawGraphsSame(histograms,
                   labels,
                   options='',
                   norm=False,
                   logy=False,
                   min_y=None,
                   max_y=None,
                   text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [ ]:
# %load samples.py
import ROOT

version = 'v54'

files = {}
file_keys = {}



class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys

class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename, 'r')#RootFile(cls.histo_filename)


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None


    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel == None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance

class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        for sample in samples:
            self.pus.append(sample.label)
        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])
        self.labels_dict = {}

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    if gen_sels is None:
                        self.data = self.data.append({'sample': 'ele',
                                                        'pu': sample.label,
                                                        'tp': tp,
                                                        'tp_sel': tp_sel,
                                                        'gen_sel': None,
                                                        'classtype': classtype,
                                                        'histo': HProxy(classtype, tp, tp_sel, None, sample.histo_file)}
                                                          , ignore_index=True)
                    else:
                        for gen_sel in gen_sels:
                            print sample, tp, tp_sel, gen_sel
                            self.data = self.data.append({'sample': 'ele',
                                                            'pu': sample.label,
                                                            'tp': tp,
                                                            'tp_sel': tp_sel,
                                                            'gen_sel': gen_sel,
                                                            'classtype': classtype,
                                                            'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)}
                                                              , ignore_index=True)



    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''
        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += '& (gen_sel == @gen_sel)'
        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)


        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo' and field[0] != 'sample'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])

#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        #print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in  histo_df['histo'].values]
        return histo, labels, text



# -------------------------------------------------------------------------
sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

samples_ele = [sample_ele_flat2to100_PU0, sample_ele_flat2to100_PU200]

# -------------------------------------------------------------------------

sample_photon_flat8to150_PU0 = Sample('photon_flat8to150_PU0', 'PU0', version)
sample_photon_flat8to150_PU200 = Sample('photon_flat8to150_PU200', 'PU0', version)

samples_photons = [sample_photon_flat8to150_PU0, sample_photon_flat8to150_PU200]


sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_photon = [sample_gPt35_PU0, sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0, sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version)
samples_nugunrates = [sample_nugunrate]


from python.selections import tp_rate_selections
from python.selections import tp_match_selections
from python.selections import gen_part_selections
from python.selections import eg_rate_selections
from python.selections import eg_qual_selections

tpsets = {'DEF': 'NNDR',
          'DEFCalib': 'NNDR Calib v1'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
# tpset_selections.update(get_label_dict(tp_match_selections))

# gen_selections.update(get_label_dict(gen_part_selections))


In [25]:
tpsets = {'EG': 'EG'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(eg_qual_selections))
gen_selections.update(get_label_dict(gen_part_selections))
gen_selections.update({'nomatch': 'nomatch'})
samples = samples_ele



## Electron flat pT gun 2 < pt < 100 GeV

In [26]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


hplot.cache_histo(classtype=histos.HistoSetEff, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


hplot.cache_histo(classtype=histos.EGHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


<__main__.Sample instance at 0x11b9e6878>
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaB
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaABC
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaA
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENPt40
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaC
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaAB
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaE
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaD
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaBC
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENPt30
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENPt20
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaBCD
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENPt10
<__main__.Sample instance at 0x11b9e6878> EG EGq2 nomatch
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GENEtaBCDE
<__main__.Sample instance at 0x11b9e6878> EG EGq2 GEN
<__main__.Sample instance

In [27]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele,PU0,EG,EGq2,GENEtaB,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b303fc8>
1,ele,PU0,EG,EGq2,GENEtaABC,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b9d1320>
2,ele,PU0,EG,EGq2,GENEtaA,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b986dd0>
3,ele,PU0,EG,EGq2,GENPt40,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x10a325e18>
4,ele,PU0,EG,EGq2,GENEtaC,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b986bd8>
5,ele,PU0,EG,EGq2,GENEtaAB,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b986ab8>
6,ele,PU0,EG,EGq2,GENEtaE,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b3ab638>
7,ele,PU0,EG,EGq2,GENEtaD,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11a8e57e8>
8,ele,PU0,EG,EGq2,GENEtaBC,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b35eab8>
9,ele,PU0,EG,EGq2,GENPt30,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11b379ab8>


In [28]:
tps = ['EG']
tp_select = ['EGq1' ,'EGq2']
gen_select = ['nomatch', 'GEN', 'GENEtaA', 'GENEtaAB', 'GENEtaABC', 'GENEtaB', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE', 'GENEtaC', 'GENEtaD', 'GENEtaE']


### Gen level particle distribution

In [9]:
for tp in ['EG']:
    for tp_sel in ['EGq1']:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_den.h_pt for his in hsets], labels, norm=True, text=text, v_lines=[1.0])
            drawSame([his.h_den.h_eta for his in hsets], labels, norm=True, text=text, v_lines=[-2.8,-2.4,-1.7,-1.52,2.8,2.4,1.7,1.52])
            drawSame([his.h_den.h_fBrem for his in hsets], labels, norm=True, text=text)
            drawSame([his.h_den.h_reachedEE for his in hsets], labels, norm=True, text=text)
            


#### 3D clusters matched to GEN

In [29]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGHistos, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_pt for his in hsets], labels, norm=True, text=text)
   

In [31]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN', 'nomatch']:
            hsets, labels, text = hplot.get_histo(histos.EGHistos, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_eta for his in hsets], labels, norm=True, text=text)
  

In [16]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.EGHistos, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_hwQual for his in hsets], labels, norm=True, text=text)
  